In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
#@title Текст заголовка по умолчанию
!pip install pyTelegramBotApi
!pip install transformers
import telebot
from typing import Tuple, List
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, TextDataset, DataCollatorForLanguageModeling,
                          AutoModelForCausalLM, BertModel,
                          Trainer, TrainingArguments,
                          TextGenerationPipeline, pipeline)
from torch.cuda import empty_cache
import os
import time
import copy

eval_batch_size : int = 16
model_name       : str = "sberbank-ai/rugpt3medium_based_on_gpt2"
split_token      : str = "<|endoftext|>"
output_dir = "/content/drive/MyDrive/GPT2 model/"
print("Введите ключ к боту")
botkey=input()
bot = telebot.TeleBot(botkey)

def predictStr(start):
    eval_model = TextGenerationPipeline(model=model, 
                                    tokenizer=tokenizer,
                                    batch_size=eval_batch_size,
                                    device=0,
                                    max_length=50)
    predictions = eval_model(start)
    print(predictions)

def predict(start, expect) -> bool:
    eval_model = TextGenerationPipeline(model=model, 
                                    tokenizer=tokenizer,
                                    batch_size=eval_batch_size,
                                    device=0,
                                    max_length=50)
    predictions = eval_model(start)
    print(expect, predictions[0]["generated_text"].split(" ")[len(start)], expect == predictions[0]["generated_text"].split(" ")[len(start)])
    try:
        if expect == predictions[0]["generated_text"].split(" ")[len(start)]:
            return True
    except Exception as e:
        return False
    return False

def load_model(path : str = model_name, resize : bool = True) -> Tuple[AutoTokenizer, AutoModelForCausalLM]:
    tokenizer = AutoTokenizer.from_pretrained(model_name, eos_token=split_token, pad_token=split_token)
    PATH = 'config.json'
    model = AutoModelForCausalLM.from_pretrained(path)
    if resize:
        model.resize_token_embeddings(len(tokenizer))
    return tokenizer, model

@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Я на связи. Напиши мне что-нибудь )')

@bot.message_handler(content_types=["text"])
def handle_text(message):
    bot.send_message(message.chat.id, 'Вы написали: ' + message.text)
    mes = message.text.replace("\n","")
    predictStr(mes)

@bot.message_handler(content_types=["document"])
def handle_document(message):
    if message.document.mime_type != 'text/plain':
        bot.send_message(message.chat.id, 'Вы отправили файл не в формате txt')
        return

    file_path = bot.get_file(message.document.file_id)
    downloaded_file = bot.download_file(file_path.file_path)
    with open(message.document.file_name, 'wb') as new_file:
        new_file.write(downloaded_file)

    new_file = open(message.document.file_name, 'r', encoding = 'utf-8')
    downloaded_file = new_file.read().lower()
    bot.send_message(message.chat.id, 'Проверка структуры')
    if(downloaded_file.find("— это") == -1):
        bot.send_message(message.chat.id, 'Отсутствует основные положения')
    if(downloaded_file.find("компани") == -1):
        bot.send_message(message.chat.id, 'Отсутствует информация о компании')
    if(downloaded_file.find("третьим лицам") == -1 and downloaded_file.find("третьи лица") == -1):
        bot.send_message(message.chat.id, 'Отсутствует информация о распространении')
    if(downloaded_file.find("срок действия") == -1 and downloaded_file.find(" хранени") == -1):
        bot.send_message(message.chat.id, 'Отсутствует информация о хранении')
    if(downloaded_file.find("обратная связь") == -1 and downloaded_file.find("контакт") == -1):
        bot.send_message(message.chat.id, 'Отсутствует информация об обратной связи')

    new_file.seek(0)
    with open(message.document.file_name.replace(".txt","") + "_edit.txt", 'w', encoding = 'utf-8') as result:
        for line in new_file:
            a = ""
            line = line.replace("\n","").split(" ")
            if(len(line) > 0):
                result.write(line[0] + " ")
                for i in range(len(line) - 1):
                    a += line[i]
                    if not predict(a, line[i + 1]):
                        result.write("*" + line[i + 1] + "* ")
                    else:
                        result.write(line[i + 1] + " ")
                    a += " "
                result.write("\n")
        
    new_file.close()
    with open(message.document.file_name.replace(".txt","") + "_edit.txt", 'rb') as new_file:
        bot.send_document(message.chat.id, new_file)
    
    os.remove(message.document.file_name)
    os.remove(message.document.file_name.replace(".txt","") + "_edit.txt")
    bot.send_message(message.chat.id, 'Обработано')

saved_model_path = "/content/drive/MyDrive/GPT2 model/checkpoint-10000"
tokenizer, model = load_model(saved_model_path, resize=False)
while True:
    #try:
        bot.polling(none_stop=True, interval=0)

    #except Exception as e:
        #print(e)  # или просто print(e) если у вас логгера нет,
        #time.sleep(1)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: ignored